In [1]:
from tkinter import *
from tkinter import ttk
from pymongo import MongoClient
from datetime import date

In [2]:
class bmiCalculator:
    
    def __init__(self):

        self.root = Tk()
        self.root.title('BMI Calculator')
        self.root.geometry('800x800')

        self.style = ttk.Style(self.root)
        self.style.theme_use('vista')

        self.bmi = DoubleVar()
        self.category = StringVar()
        self.logged = False
        self.loggedUser = None
        self.saved = [0, 0, 0, '']

        self.client = MongoClient('localhost', 27017)
        self.users = self.client.bmi.users
        self.history = self.client.bmi.history
        
        self.unameLabel = ttk.Label(self.root, text = 'User Name:')
        self.unameEntry = ttk.Entry(self.root, cursor = 'xterm', justify = CENTER)

        self.pswdLabel = ttk.Label(self.root, text = 'Password:')
        self.pswdEntry = ttk.Entry(self.root, cursor = 'xterm', justify = CENTER) 

        self.heightLabel = ttk.Label(self.root, text = 'Height (metres):'.center(10))
        self.heightEntry = ttk.Entry(self.root, cursor = 'xterm', justify = CENTER)

        self.weightLabel = ttk.Label(self.root, text = 'Weight (kilograms):'.center(10))
        self.weightEntry = ttk.Entry(self.root, cursor = 'xterm', justify = CENTER)

        self.calcButton = ttk.Button(self.root, text = 'Calculate'.center(15), cursor = 'hand2', command = self.calculate)
        self.loginButton = ttk.Button(self.root, text = 'Login'.center(15), cursor = 'hand2')
        self.historyButton = ttk.Button(self.root, text = 'History'.center(15), cursor = 'hand2', command = self.showHistory)
        self.saveButton = ttk.Button(self.root, text = 'Save'.center(15), cursor = 'hand2', command = self.save)
        self.signupButton = ttk.Button(self.root, text = 'Sign Up'.center(15), cursor = 'hand2', command = self.adduser)
        self.backButton = ttk.Button(self.root, text = 'Back'.center(15), cursor = 'hand2', command = self.calculator)

        self.histTable = ttk.Treeview(self.root, columns = (1, 2, 3, 4, 5), show = 'headings',
                                      height = 20, selectmode = 'browse')
        self.histTable.column(1, width = 110, minwidth = 20, anchor = CENTER)
        self.histTable.column(2, width = 110, minwidth = 20, anchor = CENTER)
        self.histTable.column(3, width = 110, minwidth = 20, anchor = CENTER)
        self.histTable.column(4, width = 110, minwidth = 20, anchor = CENTER)
        self.histTable.column(5, width = 110, minwidth = 20, anchor = CENTER)
        self.histTable.heading(1, text = 'Date')
        self.histTable.heading(2, text = 'Height')
        self.histTable.heading(3, text = 'Weight')
        self.histTable.heading(4, text = 'BMI')
        self.histTable.heading(5, text = 'Category')
        
        self.bmiLabel = ttk.Label(self.root)
        self.loginLabel = ttk.Label(self.root)

    def calculator(self):
        
        for widget in [self.heightEntry, self.weightEntry]:
            widget.delete(0, END)

        self.bmi.set(0)

        for widget in [self.bmiLabel, self.unameLabel, self.pswdLabel, self.unameEntry, self.pswdEntry, self.loginLabel, self.signupButton, self.backButton, self.histTable]:
            widget.place_forget()

        if all(self.saved):
            
            self.weightEntry.insert(0,self.saved[0])
            self.heightEntry.insert(0, self.saved[1])
            self.bmi.set(self.saved[2])
            self.category.set(self.saved[3])

            self.bmiLabel.config(text = 'Your BMI is {0:.2f}. You are {1}.'.format(float(self.bmi.get()), self.category.get()))
            self.bmiLabel.place(relx = 0.2, rely = 0.5)

        self.heightLabel.place(relx = 0.22, rely = 0.2)
        self.heightEntry.place(relx = 0.35, rely = 0.2)

        self.weightLabel.place(relx = 0.2, rely = 0.3)
        self.weightEntry.place(relx = 0.35, rely = 0.3)

        self.calcButton.place(relx = 0.25, rely = 0.4)
        self.loginButton.place(relx = 0.35, rely = 0.4)

        if not self.logged:
            self.loginButton.config(text = 'Login'.center(15), command = self.login)

        if self.logged:
            self.loginButton.config(text = 'Logout'.center(15), command = self.logout)
            self.saveButton.place(relx = 0.25, rely = 0.45)
            self.historyButton.place(relx = 0.35, rely = 0.45)

    def calculate(self):

        try:
            if float(self.weightEntry.get()) == 0 or float(self.heightEntry.get()) == 0:
                raise Exception
                
        except:
            self.bmiLabel.config(text = 'Invalid input.')
            self.bmiLabel.place(relx = 0.2, rely = 0.5)
            self.root.after(1000, self.bmiLabel.place_forget)
            
        else: 
            self.bmi.set(float(self.weightEntry.get())/(float(self.heightEntry.get())*float(self.heightEntry.get())))

            if self.bmi.get() < 16.5:
                self.category.set('Severely Underweight')
            elif self.bmi.get() < 18.5:
                self.category.set('Underweight')
            elif self.bmi.get() < 25:
                self.category.set('Normal Weight')
            elif self.bmi.get() < 30:                
                self.category.set('Overweight')
            else:
                self.category.set('Obese')
                
            self.bmiLabel.config(text = 'Your BMI is {0:.2f}. You are {1}.'.format(float(self.bmi.get()), self.category.get()))
            self.bmiLabel.place(relx = 0.2, rely = 0.5) 

            if not self.logged:
                self.loginLabel.config(text = 'Login to save your results!')
                self.loginLabel.place(relx = 0.2, rely = 0.55) 
                self.root.after(2000, self.loginLabel.place_forget)
                
            self.saved = [self.weightEntry.get(), self.heightEntry.get(), self.bmi.get(), self.category.get()]

    def login(self):
        
        for widget in [self.heightEntry, self.weightEntry, self.heightLabel, self.weightLabel, self.calcButton, self.bmiLabel, self.loginLabel, self.saveButton, self.historyButton]:
            widget.place_forget()

        self.unameLabel.place(relx = 0.2, rely = 0.1)
        self.unameEntry.place(relx = 0.3, rely = 0.1)
        self.unameEntry.delete(0, END)
        
        self.pswdLabel.place(relx = 0.2, rely = 0.2)
        self.pswdEntry.place(relx = 0.3, rely = 0.2)
        self.pswdEntry.delete(0, END)

        self.loginButton.config(command = self.validate)
        self.loginButton.place(relx = 0.2, rely = 0.3)

        self.signupButton.place(relx = 0.3, rely = 0.3)
        self.backButton.place(relx = 0.4, rely = 0.3)      

    def adduser(self):
        
        if self.unameEntry.get() == '' or self.pswdEntry.get() == '':
            
            self.loginLabel.config(text = 'Invalid Sign Up Attempt. Cannot leave fields empty.')
            self.loginLabel.place(relx = 0.2, rely = 0.4)
            self.root.after(1000, self.loginLabel.place_forget)            

        elif self.users.find_one({'uname': f'{self.unameEntry.get()}'}):
            
            self.loginLabel.config(text = 'This user already exists. Please try again.')
            self.loginLabel.place(relx = 0.2, rely = 0.4)
            self.root.after(1000, self.loginLabel.place_forget)

        else:
            
            self.users.insert_one({'uname': f'{self.unameEntry.get()}', 'pswd': f'{self.pswdEntry.get()}'})
            
            self.logged = True
            self.loggedUser = self.unameEntry.get()
            
            self.loginLabel.config(text = 'User Created Successfully.')
            self.loginLabel.place(relx = 0.2, rely = 0.4)
            self.root.after(1000, self.calculator)

    def validate(self):

        if self.users.find_one({'uname': f'{self.unameEntry.get()}', 'pswd': f'{self.pswdEntry.get()}'}):
            self.logged = True
            self.loggedUser = self.unameEntry.get()
            
            self.loginLabel.config(text = 'Logged In Successfully.')
            self.loginLabel.place(relx = 0.2, rely = 0.4)
            self.root.after(1000, self.calculator)
            
        else:       
            self.loginLabel.config(text = 'Login Unsuccessful. Please try again.')
            self.loginLabel.place(relx = 0.2, rely = 0.4)
            self.root.after(1000, self.loginLabel.place_forget)

    def logout(self):
        
        self.logged = False
        self.loggedUser = None

        for widget in [self.heightEntry, self.weightEntry]:
            widget.delete(0, END)

        self.bmi.set(0)
        self.saved = [0, 0, 0, '']
        
        for widget in [self.historyButton, self.saveButton, self.bmiLabel]:
            widget.place_forget()

        self.loginLabel.config(text = 'Logged Out Successfully.')
        self.loginLabel.place(relx = 0.2, rely = 0.5)
        self.root.after(1000, self.loginLabel.place_forget)

        self.loginButton.config(text = 'Login'.center(15), command = self.login)
                                
    def save(self):

        if self.logged and self.loggedUser and all(self.saved):
            self.history.insert_one({'uname':self.loggedUser, 'date':str(date.today()), 'height':round(float(self.heightEntry.get()),2), 'weight':round(float(self.weightEntry.get()),2), 'bmi':round(float(self.bmi.get()),2), 'category':self.category.get()})

            self.loginLabel.config(text = 'Your bmi has been saved successfully!')
            self.loginLabel.place(relx = 0.2, rely = 0.5)

        else:
            self.loginLabel.config(text = 'Unable to save your bmi')
            self.loginLabel.place(relx = 0.2, rely = 0.5)            
            
        self.saved = [0, 0, 0, '']
        self.root.after(1000, self.calculator)

    def showHistory(self):
        
        for widget in [self.heightEntry, self.weightEntry, self.heightLabel, self.weightLabel, self.calcButton, self.loginButton, self.bmiLabel, self.loginLabel, self.saveButton, self.historyButton]:
            widget.place_forget()

        self.histTable.place(relx = 0.1, rely = 0.1)
        self.backButton.place(relx = 0.7, rely = 0.75)

        for record in self.histTable.get_children():
            self.histTable.delete(record)

        records = self.history.find({'uname':self.loggedUser})
        for record in records:
            self.histTable.insert('', 'end', values = (record['date'], record['height'], record['weight'], record['bmi'], record['category']))    

In [3]:
if __name__ == '__main__':

    bmiCalc = bmiCalculator()
    bmiCalc.calculator()
    bmiCalc.root.mainloop()